In [3]:
import sagemaker
from sagemaker.session import TrainingInput
import numpy as np
from sagemaker.serializers import CSVSerializer
from sagemaker.pytorch import PyTorch
import pandas as pd
from sklearn.metrics import classification_report

sagemaker_session = sagemaker.Session()

bucket = 'datascience-sagemaker-fernandosousa'
prefix = 'perceptron/cobranca'

instance_type = 'ml.m5.large'

role = sagemaker.get_execution_role()

In [4]:
# preparacao
dados = pd.read_csv("s3://{}/{}".format(bucket, "Case_cobranca.csv"))
dados['CLIENTE_NOVO']   = dados['TIPO_CLIENTE'].apply(lambda x: 1 if x == 'NOVO' else 0)
dados['CLIENTE_INVESTIDOR']   = dados['TIPO_CLIENTE'].apply(lambda x: 1 if x == 'INVESTIDOR' else 0)    
dados['EMPRESTIMO_CDC']   = dados['TIPO_EMPRESTIMO'].apply(lambda x: 1 if x == 'CDC' else 0)
dados['EMPRESTIMO_PESSOAL']   = dados['TIPO_EMPRESTIMO'].apply(lambda x: 1 if x == 'PESSOAL' else 0)
dados['SEXO_M']   = dados['CD_SEXO'].apply(lambda x: 1 if x == 'M' else 0)
dados['IDADE_NORM'] = dados['IDADE'].apply(lambda x: 18 if np.isnan(x) or x < 18 else x) # mínimo
dados['IDADE_NORM'] = dados['IDADE_NORM'].apply(lambda x: 76 if x > 76 else x) # máximo
dados['IDADE_NORM'] = dados['IDADE_NORM'].apply(lambda x: (x-18)/(76-18)) # normalização entre 0 e 1
dados['QTD_DIVIDAS_NORM'] = dados['QTD_DIVIDAS'].apply(lambda x: 0. if np.isnan(x) else x/16) # normalização entre 0 e 1
dados['ALVO']   = dados['TEMP_RECUPERACAO'].apply(lambda x: 1 if x <= 90 else 0)
dados = dados.drop(['COD', 'TIPO_CLIENTE', 'TIPO_EMPRESTIMO', 'CD_SEXO', 'IDADE', 'QTD_DIVIDAS', 'TEMP_RECUPERACAO'], axis=1)


In [5]:
#salvar arquivo csv
dados.to_csv('data/train.csv')

# enviar para s3
sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)

's3://datascience-sagemaker-fernandosousa/perceptron/cobranca'

In [6]:
train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train.csv"), content_type="csv"
)
validation_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train.csv"), content_type="csv"
)

In [10]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mlp.py',
                    base_name='credito_MLP',
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters={
                        'epochs': 1000,
                        'backend': 'gloo'
                    })

In [ ]:
estimator.fit({"training": train_input, "validation": validation_input})

2022-05-08 02:24:23 Starting - Starting the training job...
2022-05-08 02:24:46 Starting - Preparing the instances for trainingProfilerReport-1651976663: InProgress
......
2022-05-08 02:25:51 Downloading - Downloading input data......
2022-05-08 02:26:46 Training - Downloading the training image.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-08 02:27:29,497 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-08 02:27:29,501 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-05-08 02:27:29,521 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-08 02:27:29,529 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-08 02:27:29,962 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-05-08 02:27:29,981 sagemaker-training-t